In [5]:
import pandas as pd
from sqlalchemy import create_engine

In [4]:
### Load csv into dataframe

In [16]:
# Load player.csv into datafram
df_players_raw = pd.read_csv('./Resources/baseball/player.csv')
df_players_raw.head()


,player_id,birth_year,birth_month,birth_day,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,name_last,name_given,weight,height,bats,throws,debut,final_game,retro_id,bbref_id
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,220.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [17]:
# Create df_player with columns of interest
df_players = df_players_raw[['player_id', 'name_first', 'name_last']]
df_players.head()

,player_id,name_first,name_last
0,aardsda01,David,Aardsma
1,aaronha01,Hank,Aaron
2,aaronto01,Tommie,Aaron
3,aasedo01,Don,Aase
4,abadan01,Andy,Abad


In [18]:
# Connect to SQL DB
connection_string = "root:chaos111@localhost/etl_project"
engine = create_engine(f"mysql://{connection_string}") 

# Confirm tables
engine.table_names()

['colleges', 'players']

In [20]:
df_players.to_sql(name='players', con=engine, if_exists='replace', index=False)

In [21]:
pd.read_sql_query('select * from players', con=engine).head()

,player_id,name_first,name_last
0,aardsda01,David,Aardsma
1,aaronha01,Hank,Aaron
2,aaronto01,Tommie,Aaron
3,aasedo01,Don,Aase
4,abadan01,Andy,Abad
